# Thermal states for 3-SAT

In [1]:
import numpy as np
# import qit
import matplotlib
import matplotlib.pyplot as plt
from random import randint
# import pycosat
import random
import scipy.linalg as la
import pandas as pd
import scipy.sparse as sparse
import scipy.sparse.linalg as lasp
from numba import jit
# for parallel computations on CPU
import pymp
import qit
import scipy

from sat import *
from qaoa import qaoa,get_params
from numpy import array

In [ ]:
%%time
N=15
beta=0.20
with pymp.Parallel(6) as pp:
    for m in pp.range(1, 20*N):
        clauses=[]
        overlap=[]
        for i in range(100):
            cnf=cnf_random_generator(N,m)
            overlap.append(thermal_overlap(cnf,N,beta))
            clauses.append(m)
        bits=[N]*100
        bet=[beta]*100
        df=pd.DataFrame(data={'bits':np.array(bits).ravel(), 
                            'beta':np.array(bet).ravel(), 
                            'clauses': np.array(clauses).ravel(), 
                            'overlap': np.array(overlap).ravel()})
        df=df[['bits','beta','clauses','overlap']]
        with open('3_SAT_cooling_15_a.csv', 'a') as f:
            df.to_csv(f, header=False)

# QAOA

In [11]:
# projection on ground_state
def projection_ground_space(cnf, n_qubits, p):
    result=0
    qaoa_state=qaoa(cnf_to_hamiltonian(cnf,n_qubits),n_qubits, p)[1]
    for i in np.where(ground_state(cnf, n_qubits) != 0)[0]:
        result+=np.abs(qaoa_state[i])**2
    return result

In [12]:
cnf_1=[np.array([ 1,  2, -6]),
 np.array([ 1, -4, -5]),
 np.array([ 1,  3, -6]),
 np.array([-1, -2, -6]),
 np.array([ 4,  5, -6])]

In [13]:
params_1=get_params(cnf_to_hamiltonian(cnf_1,6),6,20)
overlap_1=projection_ground_space(cnf_1,6,20)
print(params_1)
print(overlap_1)

[0.7404243161981697, 0.7775705695468255, 0.8187786578279733, 0.7188965854341821, 0.7814507895438692, 0.8340820742165719, 0.7083027607864854, 0.7807916428951017, 0.8392077522875999, 0.6988261041793634, 0.7955913112924764, 0.8261247194852741, 0.7040836153170253, 0.8178849837503516, 0.7980209751846071, 0.7123036523177729, 0.8308038426542279, 0.7721200722771089, 0.728850755760836, 0.8444957138872297, 0.7531678651687724, 0.7438252657893544, 0.8354543962559151, 0.7515014286641527, 0.7501788552600639, 0.8169326305992927, 0.767025182472883, 0.7574250008874013, 0.7912838120398488, 0.7915613646347153, 0.7476246906095821, 0.7676649029793906, 0.8167839505967803, 0.7436172772770399, 0.7570473310538989, 0.8373782655470808, 0.7259122225052086, 0.7692653469118514, 0.8303199569221997, 0.7170786728528618]
0.8749999997103098


In [ ]:
%%time
N=10
p=3
with pymp.Parallel(6) as pp:
    for m in pp.range(1, 15*N):
        clauses=[]
        overlap=[]
        for i in range(100):
            cnf=cnf_random_generator(N,m)
            overlap.append(projection_ground_space(cnf=cnf,n_qubits=N,p=p))
            clauses.append(m)
        bits=[N]*100
        bet=[p]*100
        df=pd.DataFrame(data={'bits':np.array(bits).ravel(), 
                            'p':np.array(bet).ravel(), 
                            'clauses': np.array(clauses).ravel(), 
                            'overlap': np.array(overlap).ravel()})
        df=df[['bits','p','clauses','overlap']]
        with open('3_SAT_qaoa_10.csv', 'a') as f:
            df.to_csv(f, header=False)